In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import os

import torch.utils.data as data
import pickle
import numpy as np
from PIL import Image


root = '.\data\cifar-10-batches-py'
class CIFAR10(data.Dataset):
    def __init__(self, root, transform=None, target_transform=None,train=True):
        self.root = root
        self.transform = transform
        self.target_transform = target_transform
        self.train = train  # training set or test set
        self.data = []
        self.targets = []
 
        if(self.train):
            choose_list=self.train_list
        else:
            choose_list=self.test_list
 
        for file_name in choose_list:
            file_path = os.path.join(self.root, file_name)
            with open(file_path, 'rb') as f:
                entry = pickle.load(f, encoding='latin1')
                self.data.append(entry['data'])
                if 'labels' in entry:
                    self.targets.extend(entry['labels'])
                else:
                    self.targets.extend(entry['fine_labels'])
        self.data = np.vstack(self.data).reshape(-1, 3, 32, 32)
        self.data = self.data.transpose((0, 2, 3, 1))  
 
    def __getitem__(self, index):
        img, target = self.data[index], self.targets[index]
        img = Image.fromarray(img)
 
        if self.transform is not None:
            img = self.transform(img)
 
        if self.target_transform is not None:
            target = self.target_transform(target)
 
        return img, target
 
    def __len__(self):
        return len(self.data)
 
    train_list = ['data_batch_1','data_batch_2','data_batch_3',
                    'data_batch_4','data_batch_5']
    test_list = ['test_batch'] 


# 
#
#
#
#
transform = transforms.Compose(
    [
     transforms.RandomHorizontalFlip(),
     transforms.RandomGrayscale(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform1 = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = CIFAR10(root, transform=transform,target_transform=None,train=True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100,
                                          shuffle=True, num_workers=2)

testset = CIFAR10(root, transform=transform1,target_transform=None,train=False)
testloader = torch.utils.data.DataLoader(testset, batch_size=50,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')





In [ ]:
class Net(nn.Module):


    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(3,64,3,padding=1)
        self.conv2 = nn.Conv2d(64,64,3,padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU()

        self.conv3 = nn.Conv2d(64,128,3,padding=1)
        self.conv4 = nn.Conv2d(128, 128, 3,padding=1)
        self.pool2 = nn.MaxPool2d(2, 2, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.relu2 = nn.ReLU()

        self.conv5 = nn.Conv2d(128,128, 3,padding=1)
        self.conv6 = nn.Conv2d(128, 128, 3,padding=1)
        self.conv7 = nn.Conv2d(128, 128, 1,padding=1)
        self.pool3 = nn.MaxPool2d(2, 2, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()

        self.conv8 = nn.Conv2d(128, 256, 3,padding=1)
        self.conv9 = nn.Conv2d(256, 256, 3, padding=1)
        self.conv10 = nn.Conv2d(256, 256, 1, padding=1)
        self.pool4 = nn.MaxPool2d(2, 2, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.relu4 = nn.ReLU()

        self.conv11 = nn.Conv2d(256, 512, 3, padding=1)
        self.conv12 = nn.Conv2d(512, 512, 3, padding=1)
        self.conv13 = nn.Conv2d(512, 512, 1, padding=1)
        self.pool5 = nn.MaxPool2d(2, 2, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.relu5 = nn.ReLU()

        self.fc14 = nn.Linear(512*4*4,1024)
        self.drop1 = nn.Dropout2d()
        self.fc15 = nn.Linear(1024,1024)
        self.drop2 = nn.Dropout2d()
        self.fc16 = nn.Linear(1024,10)


    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool1(x)
        x = self.bn1(x)
        x = self.relu1(x)


        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool2(x)
        x = self.bn2(x)
        x = self.relu2(x)

        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.pool3(x)
        x = self.bn3(x)
        x = self.relu3(x)

        x = self.conv8(x)
        x = self.conv9(x)
        x = self.conv10(x)
        x = self.pool4(x)
        x = self.bn4(x)
        x = self.relu4(x)

        x = self.conv11(x)
        x = self.conv12(x)
        x = self.conv13(x)
        x = self.pool5(x)
        x = self.bn5(x)
        x = self.relu5(x)
        # print(" x shape ",x.size())
        x = x.view(-1,512*4*4)
        x = F.relu(self.fc14(x))
        x = self.drop1(x)
        x = F.relu(self.fc15(x))
        x = self.drop2(x)
        x = self.fc16(x)

        return x

    def train_sgd(self,device):
        optimizer = optim.Adam(self.parameters(), lr=0.0001)
        print('b')

        path = 'weights.tar'
        print('b')
        initepoch = 0
        print('b')
        if os.path.exists(path) is not True:
            loss = nn.CrossEntropyLoss()
            print('b')
            # optimizer = optim.SGD(self.parameters(),lr=0.01)

        else:
            checkpoint = torch.load(path)
            print('C')
            self.load_state_dict(checkpoint['model_state_dict'])
            print('C')
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            print('C')
            initepoch = checkpoint['epoch']
            print('C')
            loss = checkpoint['loss']
            print('C')



        for epoch in range(initepoch,20):  # loop over the dataset multiple times
            timestart = time.time()
            print('D')
            running_loss = 0.0
            total = 0
            correct = 0
            print('D')
            i= 0
# trainset = CIFAR10(root, transform=transform,target_transform=None,train=True)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)
            for data in trainloader:
                # get the inputs
                print(data)
                inputs, labels = data
#                 inputs, labels = inputs.to(device),labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward + backward + optimize
                outputs = self(inputs)
                l = loss(outputs, labels)
                l.backward()
                optimizer.step()

                # print statistics
                running_loss += l.item()
                # print("i ",i)
                if i % 500 == 499:  # print every 500 mini-batches
                    print('[%d, %5d] loss: %.4f' %
                          (epoch, i, running_loss / 500))
                    running_loss = 0.0
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                    print('Accuracy of the network on the %d tran images: %.3f %%' % (total,
                            100.0 * correct / total))
                    total = 0
                    correct = 0
                    torch.save({'epoch':epoch,
                                'model_state_dict':net.state_dict(),
                                'optimizer_state_dict':optimizer.state_dict(),
                                'loss':loss
                                },path)
                i+=1
            print('epoch %d cost %3f sec' %(epoch,time.time()-timestart))
           
        print('Finished Training')

    def test(self,device):
        correct = 0
        total = 0
        with torch.no_grad():
            for data in testloader:
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                outputs = self(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the 10000 test images: %.3f %%' % (
                100.0 * correct / total))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('a')
net = Net()
print('a')
net = net.to(device)
print('a')
net.train_sgd(device)
print('a')
net.test(device)

a
a
a
b
b
b
C
C
C
C
C
Finished Training
a


In [13]:
print(trainloader.dataset[1])

(tensor([[[-0.3804, -0.3176, -0.2863,  ..., -0.1765, -0.0118,  0.2078],
         [-0.4431, -0.3961, -0.2471,  ..., -0.0196,  0.1373,  0.0980],
         [-0.4745, -0.4667, -0.3804,  ..., -0.0980,  0.0902,  0.0980],
         ...,
         [-0.2706, -0.5216, -0.6706,  ...,  0.2078,  0.2235,  0.3725],
         [ 0.0275, -0.0353, -0.1922,  ...,  0.2471,  0.2235,  0.2941],
         [ 0.1216,  0.1216,  0.1216,  ...,  0.2784,  0.2392,  0.2784]],

        [[-0.3647, -0.2941, -0.2549,  ..., -0.1843,  0.0745,  0.3882],
         [-0.4275, -0.3725, -0.2235,  ..., -0.0196,  0.2000,  0.2549],
         [-0.4588, -0.4510, -0.3569,  ..., -0.0980,  0.1451,  0.2157],
         ...,
         [-0.3490, -0.5843, -0.7333,  ...,  0.2549,  0.2078,  0.3098],
         [-0.0510, -0.1059, -0.2706,  ...,  0.2627,  0.1922,  0.2078],
         [ 0.0431,  0.0510,  0.0431,  ...,  0.2235,  0.1608,  0.1608]],

        [[-0.4510, -0.4431, -0.4431,  ..., -0.2549,  0.0667,  0.4667],
         [-0.5216, -0.5137, -0.3882,  ..., -

In [18]:
print(trainset[1][0])

tensor([[[-0.3804, -0.3176, -0.2863,  ..., -0.1922,  0.0510,  0.3412],
         [-0.4431, -0.3961, -0.2471,  ..., -0.0275,  0.1843,  0.2157],
         [-0.4745, -0.4667, -0.3804,  ..., -0.0980,  0.1294,  0.1922],
         ...,
         [-0.3176, -0.5608, -0.7098,  ...,  0.2471,  0.2157,  0.3255],
         [-0.0196, -0.0824, -0.2471,  ...,  0.2392,  0.1843,  0.2078],
         [ 0.0745,  0.0824,  0.0745,  ...,  0.2157,  0.1608,  0.1686]],

        [[-0.3804, -0.3176, -0.2863,  ..., -0.1922,  0.0510,  0.3412],
         [-0.4431, -0.3961, -0.2471,  ..., -0.0275,  0.1843,  0.2157],
         [-0.4745, -0.4667, -0.3804,  ..., -0.0980,  0.1294,  0.1922],
         ...,
         [-0.3176, -0.5608, -0.7098,  ...,  0.2471,  0.2157,  0.3255],
         [-0.0196, -0.0824, -0.2471,  ...,  0.2392,  0.1843,  0.2078],
         [ 0.0745,  0.0824,  0.0745,  ...,  0.2157,  0.1608,  0.1686]],

        [[-0.3804, -0.3176, -0.2863,  ..., -0.1922,  0.0510,  0.3412],
         [-0.4431, -0.3961, -0.2471,  ..., -0

In [22]:
import pickle
import cv2
import numpy as np
class ReadData():
    def __init__(self):
        self.label_names = None
        self.X = None
        self.Y = None
 
    def LOAD_CIFAR_DATA(self, filename):
        with open(filename, 'rb') as f:
            dataset = pickle.load(f)
            print( dataset.keys() )
            # print len(dataset['filenames'])
            X = dataset['data']
            Y = dataset['labels']
            self.X = np.reshape(X, (10000, 3, 32, 32))
            self.Y = np.array(Y)
 
    def LOAD_CIFAR_LABELS(self, filename):
        with open(filename, 'rb') as f:
            obj = pickle.load(f)
            print (obj)
            self.label_names = obj['label_names']
 
    def Read(self, imgFileName, labelFileName):
        self.LOAD_CIFAR_LABELS(filename=labelFileName)
        self.LOAD_CIFAR_DATA(filename=imgFileName)
 
    def Show(self):
        example_nums = self.X.shape[0]
        for i in range(example_nums):
            img = self.X[i]
            img_merged = cv2.merge([img[0], img[1], img[2]])
            cv2.imshow("Image", img_merged)
            print( self.label_names[self.Y[i]] )
            cv2.waitKey(0)
 
 
def main():
    readData = ReadData()
    imgFileName = './cifar-10-batches-py/data_batch_2'
    labelFileName = './cifar-10-batches-py/batches.meta'
    readData.Read(imgFileName, labelFileName)
    readData.Show()
 
 
if __name__ == '__main__':
    main()

ModuleNotFoundError: No module named 'cv2'